In [5]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN, MeanShift
from sklearn.preprocessing import MinMaxScaler
import psycopg2
from random import choice
import contextily as ctx
import numpy as np
from shapely.geometry import Point, LineString, shape
from tqdm import tqdm_notebook
import numpy as np
from multiprocessing import cpu_count, Pool
from functools import partial
import datetime
from lonelyboy.geospatial import plots as gsplt
from lonelyboy.geospatial import preprocessing as gspp
from lonelyboy.timeseries import lbtimeseries as tspp
import configparser, os
from scipy.spatial import distance_matrix
import networkx as nx

In [2]:
properties = configparser.ConfigParser()
# properties.read(os.path.join('.','sql_server.ini'))
# properties = properties['SERVER']

# host    = properties['host']
# db_name = properties['db_name']
# uname   = properties['uname']
# pw      = properties['pw']
# port    = properties['port']

host    = '195.251.230.8'
db_name = 'doi105281zenodo1167595'
uname   = 'students'
pw      = 'infol@bdbl@bs2017'
port    = '46132'

# ts>1456802710 AND ts<1456975510 

# traj_sql = 'SELECT * FROM ais_data.dynamic_ships WHERE  mmsi=226179000'
traj_sql = 'SELECT * FROM ais_data.dynamic_ships WHERE  ts>1456702710 AND ts<1456975510 '
ports_sql = 'SELECT * FROM ports.ports_of_brittany'

con = psycopg2.connect(database=db_name, user=uname, password=pw, host=host, port = port)
traj = gpd.GeoDataFrame.from_postgis(traj_sql, con, geom_col='geom' )
ports = gpd.GeoDataFrame.from_postgis(ports_sql, con, geom_col='geom' )

con.close()


ports.geom = ports.geom.apply(lambda x: x[0])

In [174]:
straj = traj.sample(frac=0.00005)

In [175]:
len(straj)

31

In [35]:
data = np.array([[0, 1], [3, 4], [3,7]])

In [21]:
data

array([[0, 1, 2],
       [3, 4, 5],
       [3, 7, 8]])

In [24]:
straj.reset_index(inplace=True)
tst = np.triu(distance_matrix(straj[['lat', 'lon']].values, straj[['lat', 'lon']].values),0)
tst[tst == 0] = np.inf

data1 = np.vstack(np.where(tst<0.3)).T

In [37]:
data

array([[0, 1],
       [3, 4],
       [3, 7]])

In [83]:
for i in connected_edges(data):
    print(i)

{0, 1}
{3, 4, 7}


In [163]:
data

array([[0, 1],
       [3, 4],
       [3, 5]])

In [169]:
index_of_cluster(3, data)

[1, 2]

In [176]:
def index_of_cluster(item, cluster_list):
    position = [ind for ind, subl in enumerate(cluster_list) if item in subl]
    if len(position)>1:
        raise ValueError
    if not position:
        return [-1]
    else:
        return position

In [177]:
lst = []

In [178]:
not lst

True

In [239]:
def connected_edges(data):
    G = nx.Graph()
    G.add_edges_from(data)

    return [list(cluster) for cluster in nx.connected_components(G)]

In [240]:
def pairs_in_radius(df, radius):
    df.reset_index(inplace=True)
    distances = np.triu(distance_matrix(df[['lat', 'lon']].values, df[['lat', 'lon']].values),0)
    distances[distances == 0] = np.inf
    print (len(distances))
    dst_hist.append(distances)

    return np.vstack(np.where(distances<=radius)).T

In [253]:
dst_hist = []
df['flock_label'] = np.nan
df =  df.groupby('ts', as_index=False).apply(get_flocks)

he
6
he
6


In [252]:
def get_flocks(timeframe):
    print('he')
    timeframe.reset_index(drop=True , inplace=True)
    data = pairs_in_radius(timeframe, 0.1)
    clusters = connected_edges(data)
    timeframe['flock_label'] = timeframe.apply( lambda x: index_of_cluster(x.name, clusters)[0], axis=1)
    return timeframe

In [257]:
straj.drop_duplicates(['mmsi'], inplace=True)

In [262]:
straj.groupby(['ts'], as_index=False).apply(lambda x: print(x))
    

              id       mmsi  status   turn  speed  course  heading       lon  \
104256  17511366  227318010    15.0  127.0    6.8   127.0      126 -4.487248   
222240  18846473  227369960     NaN    NaN    0.0    23.8      511 -4.489065   
355061  12277524  227318020    15.0 -127.0    0.0   138.0      511 -4.324250   
355062  12277525  227705102    15.0 -127.0    0.0    24.8      511 -4.496638   
355063  12277526  228037600    15.0 -127.0   10.2   138.8      511 -4.340718   
355064  12277527  227941000     7.0    0.0    0.0   329.3        9 -4.327225   

              lat          ts                          geom  flock_label  
104256  48.218180  1456787020   POINT (-4.4872484 48.21818)          NaN  
222240  48.377968  1456787020   POINT (-4.489065 48.377968)          NaN  
355061  48.098050  1456787020     POINT (-4.32425 48.09805)          NaN  
355062  48.382320  1456787020   POINT (-4.4966383 48.38232)          NaN  
355063  48.116943  1456787020  POINT (-4.3407183 48.116943)     

""


In [258]:
straj

,id,mmsi,status,turn,speed,course,heading,lon,lat,ts,geom,flock_label
104256,17511366,227318010,15.0,127.0,6.8,127.0,126,-4.487248,48.218180,1456787020,POINT (-4.4872484 48.21818),NaN
222240,18846473,227369960,NaN,NaN,0.0,23.8,511,-4.489065,48.377968,1456787020,POINT (-4.489065 48.377968),NaN
355061,12277524,227318020,15.0,-127.0,0.0,138.0,511,-4.324250,48.098050,1456787020,POINT (-4.32425 48.09805),NaN
355062,12277525,227705102,15.0,-127.0,0.0,24.8,511,-4.496638,48.382320,1456787020,POINT (-4.4966383 48.38232),NaN
355063,12277526,228037600,15.0,-127.0,10.2,138.8,511,-4.340718,48.116943,1456787020,POINT (-4.3407183 48.116943),NaN
355064,12277527,227941000,7.0,0.0,0.0,329.3,9,-4.327225,48.100100,1456787020,POINT (-4.327225 48.1001),NaN


In [233]:
import pprint
pp = pprint.PrettyPrinter(indent=8)
pp.pprint(len(dst_hist))

2


In [256]:
for _, df in traj.groupby('ts'):
    if len(df)>10:
        straj = df
        print('ok')
        break

ok


In [156]:
tmp.reset_index(inplace=True, drop=True)

In [160]:
tmp['flock_label'] = tmp.apply( lambda x: index_of_cluster(x.name, data)[0], axis=1)

IndexError: ('list index out of range', 'occurred at index 2')

In [102]:
tmp.reset_index(inplace=True, drop=True)

In [119]:
tmp.apply(lambda x : print(int(x.name)+1), axis=1)

1
2
3
4
5
6


0    None
1    None
2    None
3    None
4    None
5    None
dtype: object

In [158]:
tmp

,id,mmsi,status,turn,speed,course,heading,lon,lat,ts,geom,flock_label
0,17491273,210446000,0.0,-126.0,9.2,208.6,203,-6.697908,47.744230,1456713221,POINT (-6.6979084 47.74423),NaN
1,17491275,210446000,0.0,0.0,9.2,210.0,203,-6.698130,47.743973,1456713228,POINT (-6.69813 47.743973),NaN
2,17491438,210446000,0.0,-126.0,9.4,208.2,204,-6.744282,47.687225,1456714715,POINT (-6.744282 47.687225),NaN
3,17491442,210446000,0.0,-126.0,9.4,209.6,203,-6.744630,47.686806,1456714725,POINT (-6.74463 47.686806),NaN
4,17491671,210446000,0.0,0.0,9.4,205.5,202,-6.809920,47.605293,1456716828,POINT (-6.80992 47.605293),NaN
5,17490658,210446000,0.0,127.0,9.3,206.0,195,-6.517879,47.992603,1456706919,POINT (-6.5178785 47.992603),NaN


In [159]:
data = np.array([[0, 1], [3, 4], [3,5]])